In [2]:
# import
import datetime
import matplotlib.pyplot as plt
import matplotlib
import xarray as xr
import numpy as np
import s3fs
import fsspec
import dask
from dask.distributed import performance_report, Client, LocalCluster
import pandas as pd
import cartopy.crs as ccrs
import cartopy
from matplotlib import colors

c:\Users\yangm\anaconda3\lib\site-packages\xarray\backends\cfgrib_.py:29: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


In [3]:
from dask.distributed import Client, LocalCluster

cluster = LocalCluster()
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 6
Total threads: 24,Total memory: 31.92 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:60613,Workers: 6
Dashboard: http://127.0.0.1:8787/status,Total threads: 24
Started: Just now,Total memory: 31.92 GiB
Comm: tcp://127.0.0.1:60673,Total threads: 4
Dashboard: http://127.0.0.1:60674/status,Memory: 5.32 GiB
Nanny: tcp://127.0.0.1:60619,


In [4]:
# Make garbage collection explicit to prevent deadlocks - Adapted from:
# https://nbviewer.org/github/awslabs/amazon-asdi/blob/main/examples/dask/notebooks/era5_zarr.ipynb

import gc
from distributed import WorkerPlugin


class WorkerExplicitGC(WorkerPlugin):
    def setup(self, worker):
        self.worker = worker
        gc.disable()

    def transition(self, key, start, finish, *args, **kwargs):
        if finish == "executing":
            self.worker._throttled_gc.collect()
            if gc.isenabled():
                gc.disable()


plugin = WorkerExplicitGC()
client.register_worker_plugin(plugin)

{'tcp://127.0.0.1:60658': {'status': 'OK'},
 'tcp://127.0.0.1:60661': {'status': 'OK'},
 'tcp://127.0.0.1:60664': {'status': 'OK'},
 'tcp://127.0.0.1:60665': {'status': 'OK'},
 'tcp://127.0.0.1:60670': {'status': 'OK'},
 'tcp://127.0.0.1:60673': {'status': 'OK'}}

In [35]:
# construct filelist for yearly datasets
# define the urls
base_url = "https://rda.ucar.edu/thredds/dodsC/files/g/ds633.1_nc/e5.moda.an.sfc/"
base_url_t = "/e5.moda.an.sfc.128_034_sstk.ll025sc."
base_url3 = ".nc"

base_url1 = "https://rda.ucar.edu/thredds/dodsC/files/g/ds633.1_nc/e5.moda.fc.sfc.meanflux/"
base_url_p= "/e5.moda.fc.sfc.meanflux.235_055_mtpr.ll025sc."
base_url3 = ".nc"

# create datetimeindex
time_dim = pd.DatetimeIndex(
    [
        mon_day
        for y in range(1979, 2022)
        for mon_day in pd.date_range(f"{y}-01-01", f"{y}-12-01", freq="1MS")
    ]
)


file_list_t = []
file_list_p = []
for dt in time_dim[::12]:  # 3 months belong to the same year
    # get year and month
    year = dt.strftime("%Y")
    month = year + "010100"
    month1 = year + "120100"

    # build complete file name
    single_file_t = base_url + year + base_url_t + month + "_" + month1 + base_url3
    single_file_p = base_url1 + year + base_url_p + month + "_" + month1 + base_url3
    file_list_t.append(single_file_t)
    file_list_p.append(single_file_p)

print(len(file_list_t))
print(file_list_p[0])

43
https://rda.ucar.edu/thredds/dodsC/files/g/ds633.1_nc/e5.moda.fc.sfc.meanflux/1979/e5.moda.fc.sfc.meanflux.235_055_mtpr.ll025sc.1979010100_1979120100.nc


In [54]:
time_dim = pd.DatetimeIndex(
    [
        mon_day
        for y in range(1979, 2022)
        for mon_day in pd.date_range(f"{y}-01-01", f"{y}-12-01", freq="1MS")
    ]
)

lsm = xr.open_dataset(
                      'https://rda.ucar.edu/thredds/dodsC/files/g/ds633.0/e5.oper.invariant/197901/e5.oper.invariant.128_172_lsm.ll025sc.1979010100_1979010100.nc',
                      engine='netcdf4')

lsm = lsm['LSM'].sel(latitude=slice(65,-65),longitude=slice(120,300),time=lsm.time[0])
lsm = lsm.expand_dims(time=time_dim)
lsm.to_netcdf('data/lsm.nc')

In [60]:
# Unit: K
ds_t = xr.open_mfdataset(file_list_t, parallel=True, chunks={"time": "50MB"})
ds_t = ds_t['SSTK'].sel(latitude=slice(65,-65),longitude=slice(120,300)).compute()
ds_t_mean = ds_t.mean(dim='time')
ds_t_anom = ds_t - ds_t_mean
ds_t_anom.to_netcdf('data/t_anomaly.nc')
ds_t_anom_sea = ds_t - ds_t_mean.where(lsm==0)
ds_t_anom_sea.to_netcdf('data/t_anomaly_sea.nc')

In [65]:
# Unit: kg m**-2 s**-1 
ds_p = xr.open_mfdataset(file_list_p, parallel=True, chunks={"time": "50MB"})
ds_p = ds_p['MTPR'].sel(latitude=slice(65,-65),longitude=slice(120,300)).compute()
ds_p_mean = ds_p.mean(dim='time')
ds_p_anom = ds_p - ds_p_mean
ds_p_anom.to_netcdf('data/p_anomaly.nc')
ds_p_anom_sea = ds_p - ds_p_mean.where(lsm==0)
ds_p_anom.where(lsm==0).to_netcdf('data/p_anomaly_sea.nc')

In [68]:
p_anom_sea = xr.load_dataset('data/p_anomaly_sea.nc')
p_anom_sea


<xarray.Dataset>
Dimensions:    (latitude: 521, longitude: 721, time: 516)
Coordinates:
  * latitude   (latitude) float64 65.0 64.75 64.5 64.25 ... -64.5 -64.75 -65.0
  * longitude  (longitude) float64 120.0 120.2 120.5 120.8 ... 299.5 299.8 300.0
  * time       (time) datetime64[ns] 1979-01-01 1979-02-01 ... 2021-12-01
Data variables:
    MTPR       (time, latitude, longitude) float32 nan nan nan ... nan nan nan